# Sentence Quality Scorer

In [1]:
!pip install textacy
!python3 -m spacy download en
import textacy
import pandas as pd
from textacy.text_stats import TextStats
print("Loaded")

    100% |████████████████████████████████| 143kB 2.5MB/s ta 0:00:01
    100% |████████████████████████████████| 2.0MB 342kB/s eta 0:00:01
    100% |████████████████████████████████| 12.4MB 54kB/s  eta 0:00:01   66% |█████████████████████▍          | 8.3MB 38.7MB/s eta 0:00:01
    100% |████████████████████████████████| 31.2MB 21kB/s  eta 0:00:01
    100% |████████████████████████████████| 51kB 5.6MB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 5.7MB/s eta 0:00:01
    100% |████████████████████████████████| 235kB 2.9MB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 6.1MB/s eta 0:00:01
    100% |████████████████████████████████| 1.6MB 432kB/s eta 0:00:01
  Running setup.py bdist_wheel for python-levenshtein ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
  Running setup.py bdist_wheel for ftfy ... done
  Stored in directory: /home/jovyan/.cache/pip/wheels/37/54/00/d320239bfc8aad1

    100% |████████████████████████████████| 37.4MB 48.5MB/s ta 0:00:01    30% |█████████▉                      | 11.5MB 5.9MB/s eta 0:00:05
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

    Linking successful
    /srv/conda/lib/python3.6/site-packages/en_core_web_sm -->
    /srv/conda/lib/python3.6/site-packages/spacy/data/en

    You can now load the model via spacy.load('en')

Loaded


In [2]:
df = pd.read_csv("./abstraction_scored.csv")
df.clauses_text_final = df.clauses_text_final.apply(eval)
df.voice = df.voice.apply(eval)
df.abstraction_score = df.abstraction_score.apply(eval)
df.sample(frac = 1).head(10)

,survey_id,prompt_number,prompt_id,prompt,response,clauses_text_final,voice,idx,abstraction_score
7,3127,29,29,If my mother,"was a different lady, I would not be who I am.","[was a different lady, I would not be, who I am]","[P_bevb_x, A_def, P_bevb_x]",7,"[0.14, 0.22, 0.22]"
2,2510,33,33,When I am nervous,my hands get shaky,[my hands get shaky],[A_pron_x],2,[0.14]
0,2546,36,48,Sometimes I wish that,my mom would come home more often.,[my mom would come home more often],[A_pron_x],0,[0.25]
39,1909,25,25,My main problem is,"integrating emptiness with everyday life, find...","[integrating emptiness with making, everyday l...","[A_def, A_def, A_def]",39,"[0.14, 0.14, 0.14]"
40,1814,25,25,My main problem is,how my narccissim gets in the way of seeing th...,"[how my narccissim gets in the way of, seeing ...","[A_pron_x, A_def]",40,"[0.25, 0.14]"
17,2157,17,17,When they avoided me,hmmm... I'm stumped. I have no idea how to res...,"[hmmm, I m, I have no idea how to respond to t...","[Undefined, A_def, A_def, A_def, P_bevb_x]",17,"[0.14, 0.22, 0.22, 0.14]"
20,2837,18,42,Rules,are guard rails. It is good and helpful to hav...,"[are guard rails, It is good and helpful to ha...","[P_bevb_x, P_bevb_x, A_def, A_def, A_def, A_de...",20,"[0.12, 0.22, 0.25, 0.25, 0.12, 0.14]"
29,2374,11,39,What I like to do best is,be a conduit. There is no I.,"[be a conduit, There is no I]","[A_def, P_bevb_x]",29,"[0.25, 0.22]"
27,1883,1,1,Raising a family,is a profound gift. I am tending the future. I...,"[is a profound gift, I am tending the future, ...","[P_bevb_x, P_bevb_x, P_bevb_x, A_def, P_bevb_x...",27,"[0.14, 0.22, 0.22, 0.25, 0.25, 0.14]"
36,1823,35,35,My conscience bothers me if,I'm not placing my feet on the path with an op...,[not placing my feet on the path with an open ...,"[A_pron_x, A_def, A_pron_x, A_def, A_def, A_def]",36,"[0.25, 0.22, 0.25, 0.14, 0.14, 0.25]"


In [3]:
def score_readability(text):
    doc = textacy.Doc(text, lang = "en")
    ts = TextStats(doc)
    #print("############################################################")
    #print(text)
    #print("readability_stats = ", ts.readability_stats)
    return ts.readability_stats

df['readability_attributes_score'] = df.clauses_text_final.apply(lambda arr: [score_readability(x) for x in arr])
df['grading_level'] = df.readability_attributes_score.apply(lambda dct_arr: [round(dct['flesch_kincaid_grade_level'], 2) for dct in dct_arr])
df['reading_ease'] = df.readability_attributes_score.apply(lambda dct_arr: [round(dct['flesch_reading_ease'], 2) for dct in dct_arr])
_ = """[{'flesch_kincaid_grade_level': 0.6257142857142846, 'flesch_reading_ease': 103.04428571428573, 'smog_index': 3.1291, 'gunning_fog_index': 2.8000000000000003, 'coleman_liau_index': 2.6518669999999993, 'automated_readability_index': 0.23714285714285666, 'lix': 7.0, 'gulpease_index': 93.28571428571428, 'wiener_sachtextformel': -2.5074571428571426}]"""
del df['readability_attributes_score']
df.sample(frac = 1).head(10)

,survey_id,prompt_number,prompt_id,prompt,response,clauses_text_final,voice,idx,abstraction_score,grading_level,reading_ease
25,2276,30,30,If I were in charge,i would change that by facilitating change tha...,"[i would change that by, facilitating change i...","[A_def, A_pron_x, A_pron_x, A_def, P_bevb_x]",25,"[0.25, 0.25, 0.25, 0.25, 0.14]","[-1.84, 8.37, 9.82, 3.67, 0.63]","[117.16, 52.87, 53.12, 75.88, 103.04]"
3,2509,22,43,At times I worry about,"my own dreams, nightmares I have thousands of ...","[my own dreams nightmares, I have thousands of...","[Undefined, A_def]",3,"[0.14, 0.22]","[0.72, 0.52]","[97.03, 100.24]"
36,1823,35,35,My conscience bothers me if,I'm not placing my feet on the path with an op...,[not placing my feet on the path with an open ...,"[A_pron_x, A_def, A_pron_x, A_def, A_def, A_def]",36,"[0.25, 0.22, 0.25, 0.14, 0.14, 0.25]","[2.19, -3.01, 2.19, 3.67, 0.72, 2.48]","[102.53, 120.21, 102.53, 75.88, 97.03, 87.95]"
27,1883,1,1,Raising a family,is a profound gift. I am tending the future. I...,"[is a profound gift, I am tending the future, ...","[P_bevb_x, P_bevb_x, P_bevb_x, A_def, P_bevb_x...",27,"[0.14, 0.22, 0.22, 0.25, 0.25, 0.14]","[0.72, 2.88, 1.31, 5.25, -1.84, 2.31]","[97.03, 83.32, 90.99, 62.79, 117.16, 90.96]"
4,3356,34,47,Technology,its crazy and i wish i could keep up,"[and i wish, i could keep up]","[A_def, A_def]",4,"[0.14, 0.25]","[-2.62, -2.23]","[119.19, 118.18]"
42,1665,5,5,Being with other people,Is a mutually mutating meeting of universes,"[Is a meeting of universes, mutually mutating]","[P_bevb_x, A_def]",42,"[0.22, 0.14]","[5.24, 26.49]","[66.4, -91.29]"
46,2338,1,1,Raising a family,"Captivates my whole being, just as every windo...","[Captivates my whole being, just as every wind...","[A_pron_x, A_def, P_bevb_x]",46,"[0.25, 0.14, 0.14]","[6.62, 7.37, 2.31]","[54.73, 54.7, 90.96]"
19,2046,10,10,When people are helpless,I am inspired to reach out and be of service.,"[I am inspired to reach out and, be of service]","[P_bevb_x, A_def, A_def]",19,"[0.25, 0.25]","[0.63, 1.31]","[103.04, 90.99]"
48,1889,15,41,Privacy,the human formconsicousness separates from the...,[the human formconsicousness separates from th...,[A_def],48,[0.25],[9.08],[49.54]
28,1996,22,43,At times I worry about,how differently I see and process things from ...,"[I see and and, how differently process things...","[A_def, A_def, A_def, P_bevb_x]",28,"[0.22, 0.25, 0.25, 0.25]","[-2.23, 4.96, 2.89, 7.37]","[118.18, 75.5, 77.91, 54.7]"


In [26]:
def normalize(row, x_max, x_min, reverse_arr = False):
    if not reverse_arr:
        return [round((x - x_min)/(x_max - x_min), 2) for x in row]
    return [round((-1*x - x_min)/(x_max - x_min), 2) for x in row]

reading_ease = df['reading_ease'].tolist()
reading_ease = [j for i in reading_ease for j in i]
reading_ease = [-1*x for x in reading_ease]
x_max, x_min = max(reading_ease), min(reading_ease)
df['reading_ease_normalized'] = df['reading_ease'].apply(lambda arr : normalize(arr, x_max, x_min, reverse_arr = True))

grading_levels = df['grading_level'].tolist()
grading_levels = [j for i in grading_levels for j in i]
x_max, x_min = max(grading_levels), min(grading_levels)
df['grading_level_normalized'] = df['grading_level'].apply(lambda arr : normalize(arr, x_max, x_min, reverse_arr = False))
df.sample(frac = 1).head(10)

,survey_id,prompt_number,prompt_id,prompt,response,clauses_text_final,voice,idx,abstraction_score,grading_level,reading_ease,reading_ease_normalized,grading_level_normalized
24,2721,16,16,I feel sorry,for the pain that all humans and all beings on...,"[for the pain, that all humans and all beings ...","[Undefined, A_def, A_def]",24,"[0.14, 0.25, 0.14]","[-2.62, 7.0, 2.89]","[119.19, 75.12, 77.91]","[0.01, 0.22, 0.2]","[0.03, 0.35, 0.21]"
46,2338,1,1,Raising a family,"Captivates my whole being, just as every windo...","[Captivates my whole being, just as every wind...","[A_pron_x, A_def, P_bevb_x]",46,"[0.25, 0.14, 0.14]","[6.62, 7.37, 2.31]","[54.73, 54.7, 90.96]","[0.31, 0.31, 0.14]","[0.34, 0.36, 0.19]"
28,1996,22,43,At times I worry about,how differently I see and process things from ...,"[I see and and, how differently process things...","[A_def, A_def, A_def, P_bevb_x]",28,"[0.22, 0.25, 0.25, 0.25]","[-2.23, 4.96, 2.89, 7.37]","[118.18, 75.5, 77.91, 54.7]","[0.01, 0.22, 0.2, 0.31]","[0.04, 0.28, 0.21, 0.36]"
48,1889,15,41,Privacy,the human formconsicousness separates from the...,[the human formconsicousness separates from th...,[A_def],48,[0.25],[9.08],[49.54],[0.34],[0.42]
33,2404,14,14,The past,is a construct to give form and structure to t...,"[is a construct, to give form and structure to...","[P_bevb_x, A_def]",33,"[0.25, 0.25]","[1.31, 2.34]","[90.99, 94.3]","[0.14, 0.13]","[0.16, 0.19]"
40,1814,25,25,My main problem is,how my narccissim gets in the way of seeing th...,"[how my narccissim gets in the way of, seeing ...","[A_pron_x, A_def]",40,"[0.25, 0.14]","[3.76, 3.76]","[82.39, 82.39]","[0.18, 0.18]","[0.24, 0.24]"
14,1529,10,10,When people are helpless,At times I try to find other ways of doing thi...,"[At times, I try to find other ways of, doing ...","[Undefined, A_def, A_def, A_def, A_def, A_def,...",14,"[0.11, 0.22, 0.14, 0.22, 0.12]","[-3.01, 0.63, 2.89, 5.68, -2.62]","[120.21, 103.04, 77.91, 66.79, 119.19]","[0.0, 0.09, 0.2, 0.26, 0.01]","[0.01, 0.13, 0.21, 0.3, 0.03]"
6,1869,25,25,My main problem is,a feeling that I am not entitled,"[a feeling, that I am not entitled]","[Undefined, P_bevb_x]",6,"[0.14, 0.22]","[2.89, 2.88]","[77.91, 83.32]","[0.2, 0.18]","[0.21, 0.21]"
11,2407,17,17,When they avoided me,I sought to find out 'why?' . Knowing the 'why...,"[I sought to find out, why Knowing, the why ca...","[A_def, A_def, A_def, A_def]",11,"[0.22, 0.14, 0.25]","[-1.84, 2.89, 4.0]","[117.16, 77.91, 78.87]","[0.02, 0.2, 0.2]","[0.05, 0.21, 0.25]"
41,1805,4,37,"These days, work",Is a rainbow of consciousness manifesting thro...,"[a rainbow of consciousness, manifesting throu...","[Undefined, A_def, A_def]",41,"[0.25, 0.25, 0.25]","[6.62, 12.32, -2.62]","[54.73, 15.64, 119.19]","[0.31, 0.5, 0.01]","[0.34, 0.53, 0.03]"


In [28]:
df.to_csv("readability_scored.csv", index = False)

In [31]:
reading_ease = df['reading_ease_normalized'].tolist()
reading_ease = [j for i in reading_ease for j in i]
grade = df['grading_level_normalized'].tolist()
grade = [j for i in grade for j in i]
max(reading_ease), min(reading_ease), max(grade), min(grade)

(1.0, 0.0, 1.0, 0.0)